# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [153]:
def get_gini(df, label):
    total = len(df)
    cnt_y = len(df[df[label]=='yes'])
    gini = 1- (cnt_y/total)**2 - ((total-cnt_y)/total)**2
    
    return gini    

In [19]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [24]:
from itertools import combinations  #combination 사용을 위한 import

In [68]:


def get_binary_split(df, attribute):
    lis = df[attribute].unique()
    
    c = []
    result = []
    for li in lis:
        result.append([li])
    for i in range(2,len(lis)):
        comb = list(combinations(lis,i))
        for i in range(len(comb)):
            result.append(list(comb[i]))
    return result

In [6]:
get_binary_split(pd_data,"income")

[['high'],
 ['low'],
 ['medium'],
 ['high', 'low'],
 ['high', 'medium'],
 ['low', 'medium']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [210]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    total = len(df)
    binary = get_binary_split(df,attribute)
    cnt = []
    for b in binary:
        temp = pd.DataFrame()
        name = ""
        for f in b:
            temp = pd.concat([temp, df[df[attribute]==f]])
            name+=f+"_"
        
        gini = len(temp)/total*get_gini(temp,label)+ (total-len(temp))/total*get_gini(df.drop(temp.index, axis =0),label)
        result[name[:-1]] = gini
    return result

In [209]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 보여주세요.

In [156]:
min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())

('high', 0.4428571428571429)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [177]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    _ , gini = (min(get_attribute_gini_index(pd_data, feature, label).items()))
    print("Minimum Gini index of {0} : {1:0.4f}".format(feature,float(gini)))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [179]:
#gini index가 가장 낮은 age 선택
item, gini  = min(get_attribute_gini_index(pd_data,'age', 'class_buys_computer').items())
print(item) 

middle_aged


In [190]:
#middle_aged / Youth, senior로 분리
left_df = pd_data[pd_data['age']=='middle_aged'].drop('age',axis =1)
right_df = pd_data[pd_data['age']!= 'middle_aged'].drop('age', axis =1)

In [222]:

def get_gini_index(df,dropped_feature, criteria):
    # 변수명 중 마지막에 위치한 label 컬럼 얻기
    label = df.columns[-1]
    # label 변수를 제외한 변수명 얻기
    features = list(df.columns[:-1])
    for feature in features:
        _ , gini = (min(get_attribute_gini_index(df, feature, label).items()))
        print("{0} : {1}, Minimum Gini index of {2} : {3:0.4f}".format(dropped_feature, criteria, feature,float(gini)))

In [224]:
# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)

get_gini_index(left_df,"Age",item)
print("#############################################################")
get_gini_index(right_df,"Age",item)


Age : middle_aged, Minimum Gini index of income : 0.0000
Age : middle_aged, Minimum Gini index of student : 0.0000
Age : middle_aged, Minimum Gini index of credit_rating : 0.0000
#############################################################
Age : middle_aged, Minimum Gini index of income : 0.3750
Age : middle_aged, Minimum Gini index of student : 0.3200
Age : middle_aged, Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

In [253]:
import math 
def getEntropy(df, feature) :
    total_cnt = len(df[feature])
    entropy = 0
    for var,count in df[feature].value_counts().items():
        entropy += (-(count/total_cnt)*math.log2(count/total_cnt))
    return(entropy)

In [254]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [275]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    result = {}
    for column in columns:
        total = len(df[column])
        gain = 0
        for var, count in df[column].value_counts().items():
            gain += (count/total*getEntropy(df[df[column]==var], feature))
        result[column] = (info_D -gain)
    return(result)

In [276]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}